<a href="https://colab.research.google.com/github/Gaukhar-ai/working-in-pairs/blob/master/Copy_of_Day_31_Lecture_2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline
import warnings
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

import math
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def print_vif(x):                  
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():                                #what's warnings for? the whole this function
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")

In [3]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
wine.shape

(1599, 12)

Recall that we need to check for missing data.

In [5]:
# answer below:
wine.isnull().sum()


fixed.acidity           0
volatile.acidity        0
citric.acid             0
residual.sugar          0
chlorides               0
free.sulfur.dioxide     0
total.sulfur.dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [6]:
wine.dtypes

fixed.acidity           float64
volatile.acidity        float64
citric.acid             float64
residual.sugar          float64
chlorides               float64
free.sulfur.dioxide     float64
total.sulfur.dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [7]:
# answer below
wine["quality"] = wine['quality'] > 5
wine['quality'] = wine['quality'].astype(int)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


Create a train test split with 20% of the data in the test subsample.

In [8]:
print_vif(wine.drop(columns=["quality"]))


VIF results
-------------------------------
const                   1.710538e+06
fixed.acidity           7.767512e+00
volatile.acidity        1.789390e+00
citric.acid             3.128022e+00
residual.sugar          1.702588e+00
chlorides               1.481932e+00
free.sulfur.dioxide     1.963019e+00
total.sulfur.dioxide    2.186813e+00
density                 6.343760e+00
pH                      3.329732e+00
sulphates               1.429434e+00
alcohol                 3.031160e+00
dtype: float64
-------------------------------



In [9]:
X = wine.drop(columns="quality")
y = wine["quality"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
test_size=0.20, random_state=55)


Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [10]:
from sklearn.neighbors import KNeighborsClassifier 

model = KNeighborsClassifier(5)
model.fit(X_train, y_train)

model.score(X_train, y_train)

0.7787333854573886

In [11]:
model.score(X_test, y_test)

0.628125

Scale only the independent variables using the minmax scaler.

In [17]:
# answer below:
from sklearn.preprocessing import MinMaxScaler, StandardScaler 

scaler = MinMaxScaler()
scaler = StandardScaler()                 #we have 2 scalers here, 


In [18]:
scaler.fit(X_train)                                #how do we know which one we're using?

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
import numpy as np
np.sqrt(26)


5.0990195135927845

In [30]:
a = np.array([0,0,0])
b = np.array([3,4,0])
a - b

array([-3, -4,  0])

In [31]:
c = (a - b)*(a-b) #square

In [32]:
sum(c)

25

In [33]:
np.sqrt(sum(c))  #can use any number not only on a, b above. #5 is the distance between a and b points. 

5.0

In [19]:
model = KNeighborsClassifier(5)
model.fit(X_train, y_train)

model.score(X_train, y_train)

0.8193901485535575

In [20]:
model.score(X_test, y_test)

0.7125

When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [34]:

model = KNeighborsClassifier() # or (weights = 'distance')
model.fit(X_train, y_train)
model.predict(X_train[[0], :])

array([0])